In [1]:
import os
import subprocess
import datetime as dt
import itertools
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

In [2]:
years=[1991,2020]
clim_years=[1991,2020]
basepath='/space/hall5/sitestore/eccc/crd/ccrn/users/reo000/work/MHW/OISST/'
fname='oisst-avhrr-v02r01.regridded1x1g2.monthly.1991_2020.nc'
f0=os.path.join(basepath,fname)

1. Calculate climatology

In [3]:
# open sst data
fin=xr.open_dataset(f0)

In [4]:
fin.time.values[0]

numpy.datetime64('1991-01-16T12:00:00.000000000')

In [5]:
# Find year and month (time is in months since 1960-1-1)
time=fin.S.values
yy=np.array([int(el/12)+1991 for el in np.arange(0,len(time))])
mm=np.array([int(el%12)+1 for el in np.arange(0,len(time))])

In [6]:
yy

array([1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991, 1991,
       1991, 1992, 1992, 1992, 1992, 1992, 1992, 1992, 1992, 1992, 1992,
       1992, 1992, 1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993, 1993,
       1993, 1993, 1993, 1994, 1994, 1994, 1994, 1994, 1994, 1994, 1994,
       1994, 1994, 1994, 1994, 1995, 1995, 1995, 1995, 1995, 1995, 1995,
       1995, 1995, 1995, 1995, 1995, 1996, 1996, 1996, 1996, 1996, 1996,
       1996, 1996, 1996, 1996, 1996, 1996, 1997, 1997, 1997, 1997, 1997,
       1997, 1997, 1997, 1997, 1997, 1997, 1997, 1998, 1998, 1998, 1998,
       1998, 1998, 1998, 1998, 1998, 1998, 1998, 1998, 1999, 1999, 1999,
       1999, 1999, 1999, 1999, 1999, 1999, 1999, 1999, 1999, 2000, 2000,
       2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2000, 2001,
       2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001, 2001,
       2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002, 2002,
       2002, 2003, 2003, 2003, 2003, 2003, 2003, 20

In [7]:
sst_clim=np.empty((12,len(fin.lat.values),len(fin.lon.values)))
# Calculate climatology
for im in range(1,13):
    ind = (mm==im) & (yy>=clim_years[0]) & (yy<=clim_years[1])
    sstsel=fin.sst[ind,:,:].data
    sst_clim[im-1,:,:]=sstsel.mean(axis=0)

# Save to file
f_clim=basepath+f'sst_climatology_oisst-avhrr-v02r01.regridded1x1g2.monthly.{years[0]}_{years[-1]}.nc'
xout=xr.Dataset(data_vars={'lon':(['X',],fin.lon.values),
                           'lat':(['Y',],fin.lat.values),
                        'sst_clim':(['Mon','Y','X'],sst_clim)},
            coords=dict(X=fin.X,Y=fin.Y,Mon=("Mon",np.arange(1,13))),)
xout.to_netcdf(f_clim,mode='w')
fin.close()
print('Done\n\n')

Done




2. Calculate anomalies

In [8]:
# Load climatology
fclim = xr.open_dataset(f_clim,decode_times=False)
sst_clim = fclim.sst_clim.values

# Load obs
fin = xr.open_dataset(f0,decode_times=False)
sst=fin.sst.values
time=fin.time2.values
lon=fin.lon.values
lat=fin.lat.values

# get years and months; time period is already constrained
yy=np.array([int(el/12)+1991 for el in np.arange(0,len(time))])
mm=np.array([int(el%12)+1 for el in np.arange(0,len(time))])

# Loop through time and compute anomalies
nt = len(yy)
sst_an=np.empty(np.shape(sst))
for it in range(0,nt):
    sst_an[it,...] = sst[it,...] - sst_clim[mm[it]-1,...]

# Save to file
f_anom = basepath+f'sst_anomaly_oisst-avhrr-v02r01.regridded1x1g2.monthly.{years[0]}_{years[-1]}.nc'
year = yy;
month = mm;
xout=xr.Dataset(data_vars={'lon':(['X',],lon),
                        'lat':(['Y',],lat),
                        'time':(['S'],time),
                        'year':(['S'],year),
                        'month':(['S'],month),
                        'sst_an':(['S','Y','X'],sst_an)},
            coords=dict(X=fin.X,Y=fin.Y,S=("S",time)),)
xout.to_netcdf(f_anom,mode='w')
fin.close()
fclim.close()
del sst;del sst_an;del time;del year;del month

3. Detrend anomalies

In [9]:
def _detrend(data):
    # remove trend along first dimension assuming evenly spaced data along that dimension
    # reshape to 2-d array to get fit, then transform fit back to original shape
    # return data with trend subtracted
    data=np.asarray(data)
    dshape = data.shape
    N=dshape[0]
    X=np.concatenate([np.ones((N,1)), np.expand_dims(np.arange(0,N),1)],1)
    M=np.prod(dshape,axis=0) // N # // is floor division
    newdata = np.reshape(data,(N, M)) 
    newdata = newdata.copy() # make sure a copy has been created
    # check there aren't extraneous nan's (besides fully masked land cells)
    if set(np.unique(np.sum(np.sum(np.isnan(fin.sst_an.data),axis=1),axis=0)))==set([0,np.prod(fin.sst_an.data.shape[:2])]):
        b, res, p, svs = np.linalg.lstsq(X,newdata,rcond=None)
    else: # extra nan's present; trigger annoying slow loop
        print('entering NaN loop')
        b=-9*np.ones((2,M))
        for ix in range(0,M):
            yy=newdata[:,ix]
            ind=~np.isnan(yy)
            xx=X[ind,:]
            yy=yy[ind].reshape((np.sum(ind),1))
            bb, res, p, svs = np.linalg.lstsq(xx,yy,rcond=None)
            b[:,ix]=bb[:,0]
        assert np.sum(b==-9)==0
    bshp = tuple([2]+list(dshape)[1:])
    b=np.reshape(b,bshp)
    trnd=np.arange(0,N).reshape((N,)+tuple(np.ones(len(dshape)-1,dtype=int))) * b[1,...].reshape((1,)+np.shape(b[1,...]))+\
                b[0,...].reshape((1,)+np.shape(b[0,:,:]))
    return data-trnd

In [10]:
# Load anomalies
fin=xr.open_dataset(f_anom,decode_times=False)

# Detrend
sst_an_dt = _detrend(fin.sst_an.data)

# Save to file
f_anom_dt = basepath+f'sst_anomaly_detrended_oisst-avhrr-v02r01.regridded1x1g2.monthly.{years[0]}_{years[-1]}.nc'

xout=xr.Dataset(data_vars={'lon':(['X',],fin.lon.values),
                        'lat':(['Y',],fin.lat.values),
                        'time':(['S'],fin.time.values),
                        'year':(['S'],fin.year.values),
                        'month':(['S'],fin.month.values),
                        'sst_an_dt':(['S','Y','X'],sst_an_dt)},
            coords=dict(X=fin.X,Y=fin.Y,S=fin.S),)
xout.to_netcdf(f_anom_dt,mode='w')
fin.close()
del sst_an_dt;


entering NaN loop


4. Calculate MHWs

In [11]:
for is_detrend in (True,False):
    print(is_detrend)
    # Load anomalies        
    if is_detrend:
        fin=xr.open_dataset(f_anom_dt)
        sst_an=fin.sst_an_dt.values
    else:
        fin=xr.open_dataset(f_anom)
        sst_an=fin.sst_an.values
    month=fin.month.values
    year=fin.year.values
    
    # Loop through month and compute sst anomaly thresholds for MHWs
    # Thresholds are computed with a 3-month moving window
    sst_an_thr=np.zeros((12,len(fin.Y),len(fin.X)))
    for ii in range(1,13): # month
        if ii==1:
            tind = ((month==12) | (month<=2)) & (year>=clim_years[0]) & (year<=clim_years[1])
        elif ii==12:
            tind = ((month==1) | (month>=11)) & (year>=clim_years[0]) & (year<=clim_years[1])
        else:
            tind = (month>=ii-1) & (month<=ii+1) & (year>=clim_years[0]) & (year<=clim_years[1])
        tmp = sst_an[tind,...]
        sst_an_thr[ii-1,...] = np.quantile(tmp,0.9,axis=0)
    
    # Find points that exceed thresholds
    is_mhw=np.zeros(np.shape(sst_an))
    for ii in range(0,np.shape(sst_an)[0]):
        mm=month[ii]
        is_mhw[ii,...]=np.where(sst_an[ii,...]>np.expand_dims(sst_an_thr[mm-1,...],0),1,0)
    
    # Save to file
    if is_detrend:
        f_mhw = basepath+f'mhw_detrended_oisst-avhrr-v02r01.regridded1x1g2.monthly.{years[0]}_{years[-1]}.nc'
    else:
        f_mhw = basepath+f'mhw_oisst-avhrr-v02r01.regridded1x1g2.monthly.{years[0]}_{years[-1]}.nc'
    xout=xr.Dataset(data_vars={'lon':(['X',],fin.lon.values),
                        'lat':(['Y',],fin.lat.values),
                        'time':(['S'],fin.time.values),
                        'year':(['S'],fin.year.values),
                        'month':(['S'],fin.month.values),
                        'sst_an_thr':(['Mon','Y','X'],sst_an_thr),
                        'is_mhw':(['S','Y','X'],is_mhw),},
            coords=dict(X=fin.X,Y=fin.Y,S=fin.S,Mon=("Mon",np.arange(1,13))),)
    xout.to_netcdf(f_mhw,mode='w')
    fin.close()
    del sst_an_thr; del is_mhw; del sst_an;
print('\nDone\n\n')

True
False

Done


